In [20]:
import sqlite3
conn = sqlite3.connect(r"C:\Users\oisin\Code\CS185CProject\FPA_FOD_20170508.sqlite")
c = conn.cursor()
c.execute(''' SELECT *, (LATITUDE * LATITUDE) as val1, (LONGITUDE * LONGITUDE) as val2 FROM Fires WHERE STATE = 'CA' LIMIT 1; ''')
rows = c.fetchall()

[(1, 1, 'FS-1418826', 'FED', 'FS-FIRESTAT', 'FS', 'USCAPNF', 'Plumas National Forest', '0511', 'Plumas National Forest', '1', 'PNF-47', 'BJ8K', 'FOUNTAIN', None, None, None, None, None, 2005, 2453403.5, 33, '1300', 9.0, 'Miscellaneous', 2453403.5, 33, '1730', 0.1, 'A', 40.03694444, -121.00583333, 5.0, 'USFS', 'CA', '63', '063', 'Plumas', b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@|\x01\x00\x00\x00\xe8d\xc2\x92_@^\xc0\xe0\xc8l\x98\xba\x04D@\xfe', 1602.956920091647, 14642.41169988774)]


In [189]:
import pandas as pd
import numpy as np
from scipy import spatial

In [26]:
df = pd.read_sql_query("SELECT * FROM Fires", conn)

In [129]:
df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
5,6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.635278,-120.103611,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...
6,7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.688333,-120.153333,5.0,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...
7,8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,40.968056,-122.433889,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...
8,9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,41.233611,-122.283333,13.0,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\..."
9,10,10,FS-1418859,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.548333,-120.149167,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\t^\xc0\...


In [28]:
station_df = pd.read_sql_query("""SELECT * FROM station_data WHERE state='CA'""", conn)

In [186]:
def get_closest_station(row):
#     print(row[['LATITUDE', 'LONGITUDE']])
    coords = np.array(row[['LATITUDE', 'LONGITUDE']]) ** 2
#     print(coords)
#     print((station_df[['lat', 'lon']]).pow(2) - coords)
    idx = ((station_df[['lat', 'lon']]).pow(2) - coords).apply(np.linalg.norm, axis=1).idxmin()
#     print(idx)
    return station_df.loc[idx]['usaf']

In [309]:
df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
5,6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.635278,-120.103611,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...
6,7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.688333,-120.153333,5.0,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...
7,8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,40.968056,-122.433889,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...
8,9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,41.233611,-122.283333,13.0,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\..."
9,10,10,FS-1418859,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.548333,-120.149167,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\t^\xc0\...


In [308]:
tree = spatial.KDTree(np.array(station_df[['lat', 'lon']]))

In [319]:
_, idx = tree.query(df[['LATITUDE', 'LONGITUDE']])

In [323]:
closest_station = station_df.iloc[idx]

*https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.4/_metadata_RDS-2013-0009.4.html* <br>
FIRE_SIZE_CLASS = Code for fire size based on the number of acres within the final fire perimeter expenditures (A=greater than 0 but less than or equal to 0.25 acres, B=0.26-9.9 acres, C=10.0-99.9 acres, D=100-299 acres, E=300 to 999 acres, F=1000 to 4999 acres, and G=5000+ acres).

In [424]:
new_df = df.drop(['OBJECTID', 'FOD_ID', 'Shape', 'FIPS_CODE', 
         'LOCAL_INCIDENT_ID', 'LOCAL_FIRE_REPORT_ID', 
         'FPA_ID', 'SOURCE_SYSTEM_TYPE','SOURCE_SYSTEM', 
         'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 
         'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
        'FIRE_CODE', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 
         'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'STAT_CAUSE_CODE', 
         'COUNTY'],
        axis=1)

In [425]:
closest_station = closest_station.reset_index().drop(
    ['level_0', 'index', 'wban', 'fips', 'call'], axis=1)

KeyError: "['level_0' 'wban' 'fips' 'call'] not found in axis"

In [384]:
closest_station = closest_station.rename(columns=dict(zip(closest_station.columns, ["CS-" + c.upper() for c in closest_station.columns])))

In [418]:
closest_station

,CS-USAF,CS-NAME,CS-COUNTRY,CS-STATE,CS-LAT,CS-LON,CS-ELEV,CS-BEGIN,CS-END
0,725837,SUSANVILLE MUNI,US,CA,40.383,-120.567,12650,19770301,19960718
1,745010,FRESHPOND,US,CA,38.750,-120.517,11470,None,None
2,720645,PLACERVILLE AIRPORT,US,CA,38.717,-120.750,7870,20090711,20120618
3,725847,LAKE TAHOE,US,CA,38.883,-120.000,19090,19730101,19991231
4,725847,LAKE TAHOE,US,CA,38.883,-120.000,19090,19730101,19991231
5,725847,LAKE TAHOE,US,CA,38.883,-120.000,19090,19730101,19991231
6,725847,LAKE TAHOE,US,CA,38.883,-120.000,19090,19730101,19991231
7,721027,TRINITY CENTER,US,CA,40.983,-122.694,7280,20120302,20120618
8,725957,MOUNT SHASTA,US,CA,41.317,-122.317,10770,20000101,20031231
9,725847,LAKE TAHOE,US,CA,38.883,-120.000,19090,19730101,19991231


In [426]:
total_data = pd.concat([new_df, closest_station], axis=1)

In [430]:
total_data[total_data['STATE'] == 'CA'].to_sql('aggregate_data', con=conn)